In [147]:
import re
import pandas as pd
import requests
import json
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer,TweetTokenizer
from nltk.corpus import stopwords
from nltk import download
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

download('punkt')
download('stopwords')

url = 'https://raw.githubusercontent.com/marobinette/pocs/main/project/comedy_data_10_31.csv'

df = pd.read_csv(url)
df = df[['title', 'length', 'transcript']]

def trimTitle(title):
    trimmedTitle = title.split('–')[0]
    # remove whitespace at the end
    trimmedTitle = trimmedTitle.rstrip()
    return trimmedTitle

# apply trimTitle to the title column
df['title'] = df['title'].apply(trimTitle)  

def export_comedian_word_data(word_data, comedian):
    """
    Exports the word data for a specific comedian to a CSV file.
    
    Parameters:
    - word_data (DataFrame): DataFrame containing words, counts, total unique words, and probabilities.
    - comedian (str): Name of the comedian to label the CSV file.
    """
    word_data = word_data.rename(columns={'word': 'types'})
    word_data = word_data[['types', 'counts', 'total_unique', 'probs']]    
    filename = f"{comedian}.csv"
    word_data.to_csv(filename, index=False)
    print(f"Data exported to {filename} successfully.")

def get_word_data(words):
    """
    Generates word frequency data including counts and probabilities.
    
    Parameters:
    - words (list): List of words to analyze for frequency and probability.
    
    Returns:
    - DataFrame: DataFrame with word counts, probabilities, and total unique word count.
    """
    word_data = pd.Series(words).value_counts().reset_index()
    word_data.columns = ['word', 'counts']        
    word_data['probs'] = word_data['counts'] / word_data['counts'].sum()    
    word_data['total_unique'] = len(word_data)
    word_data = word_data.sort_values(by='counts', ascending=False).reset_index(drop=True)
    
    return word_data

def tokenize_words(text, remove_stop_words=False):
    """
    Tokenizes the text into words, with an option to remove stopwords.

    Parameters:
    - text (str or list): Text or list of strings to tokenize.
    - remove_stop_words (bool): Whether to remove stopwords from the tokens.

    Returns:
    - list: List of tokenized words.
    """
    if isinstance(text, list):
        text = ' '.join(text)

    # Tokenize text while preserving contractions.
    words = word_tokenize(text)

    # Remove all words in brackets or punctuation.
    tokenizer = TweetTokenizer()
    words = tokenizer.tokenize(text)
    words = [re.sub(r"[^\w\s]", '', word) for word in words]

    # Filter out empty strings resulting from punctuation removal.
    words = [word for word in words if word]

    # Remove one-letter words except "I", "a", or "A"
    words = [word for word in words if len(word) > 1 or word in ['I', 'a', 'A']]

    # Remove stopwords if the option is selected.
    if remove_stop_words:
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word.lower() not in stop_words]

    return words


def calculate_innovation_rate(words):
    """
    Calculates the innovation rate as the ratio of unique words to the total words.
    
    Parameters:
    - words (list): List of tokenized words.
    
    Returns:
    - float: Innovation rate.
    """
    unique_words = set(words)
    print(f"Unique words: {len(unique_words)}")
    total_words = len(words)
    print(f"Total words: {total_words}")
    innovation_rate = len(unique_words) / total_words if total_words > 0 else 0
    return innovation_rate

def get_innovation_rate_data(df):
    """
    Returns a DataFrame with title, innovation rate, and length of set for each comedian.
    """
    titles = []
    lengths = []  # Renamed to avoid overwriting
    innovation_rates = []

    for idx, row in df.iterrows():
        title = row['title']
        length = row['length']  # Length of the set
        transcript = row['transcript']
        
        words = tokenize_words(transcript, remove_stop_words=False)
        innovation_rate = calculate_innovation_rate(words) * 100  # Convert to percentage
        
        # Append data to lists
        titles.append(title)
        lengths.append(length)
        innovation_rates.append(innovation_rate)
    
    # Create a new DataFrame with title, length, and innovation_rate columns
    innovation_rate_df = pd.DataFrame({
        'title': titles,
        'length': lengths,  # Corrected list name here
        'innovation_rate': innovation_rates
    })

    return innovation_rate_df


def plot_zipf_distribution(df, title):
    """
    Plots the Zipf distribution for the given comedian's transcript.
    
    Parameters:
    - df (DataFrame): DataFrame containing comedy data with titles and transcripts.
    - title (str): The comedian's name for which to plot the distribution.
    """
    # Filter the dataframe for the given comedian
    transcript = df[df['title'] == title]['transcript'].values[0]
    
    # Tokenize the words
    words = tokenize_words(transcript, remove_stop_words=False)
    
    # Generate word frequency data
    word_data = get_word_data(words)
    
    # Rank the words by frequency
    word_data['rank'] = np.arange(1, len(word_data) + 1)
    
    # Log-transform rank and frequency
    log_rank = np.log10(word_data['rank'])
    log_freq = np.log10(word_data['counts'])
    
    # Perform linear regression to calculate the slope
    slope, intercept, r_value, p_value, std_err = linregress(log_rank, log_freq)
    
    # Plot the Zipf distribution (log-log scale)
    plt.figure(figsize=(10, 6))
    plt.loglog(word_data['rank'], word_data['counts'], marker=".", linestyle='none', color="grey", label=f'(slope={slope:.2f})')
    
    plt.xlabel('Rank (log scale)')
    plt.ylabel('Frequency (log scale)')
    plt.title(f'Zipf Law - {title}')
    plt.legend()
    plt.show()

def plot_innovation_rate(df):
    plt.figure(figsize=(10, 6))
    plt.scatter(innovation_rate_df['length'], innovation_rate_df['innovation_rate'], alpha=0.7)
    plt.xlabel('Length of Set')
    plt.ylabel('Innovation Rate (%)')
    plt.title('Innovation Rate vs. Length of Set')
    plt.tight_layout()
    plt.show()

def get_innovation_rate_data_with_word_count(df):
    """
    Returns a DataFrame with title, innovation rate, and word count of each comedian's set.
    """
    titles = []
    word_counts = []
    innovation_rates = []

    for idx, row in df.iterrows():
        title = row['title']
        transcript = row['transcript']
        
        words = tokenize_words(transcript, remove_stop_words=False)
        word_count = len(words)
        innovation_rate = calculate_innovation_rate(words) * 100  # Convert to percentage
        
        # Append data to lists
        titles.append(title)
        word_counts.append(word_count)
        innovation_rates.append(innovation_rate)
    
    # Create a new DataFrame with title, word_count, and innovation_rate columns
    innovation_rate_df = pd.DataFrame({
        'title': titles,
        'word_count': word_counts,
        'innovation_rate': innovation_rates
    })

    return innovation_rate_df

def plot_innovation_rate_by_word_count(df):
    """
    Plots innovation rate against the number of words in each transcript.
    
    Parameters:
    - df (DataFrame): DataFrame containing title, word_count, and innovation_rate.
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df['word_count'], df['innovation_rate'], alpha=0.7)
    plt.xlabel('Transcript Length (Word Count)')
    plt.ylabel('Innovation Rate (%)')
    plt.title('Innovation Rate vs. Transcript Length')
    plt.tight_layout()
    plt.show()

def calculate_vocabulary_growth(words):
    """
    Calculates vocabulary size as a function of word count for a given transcript.
    Returns a list of tuples where each tuple contains the count of words and unique words.
    """
    vocab_growth = []
    unique_words = set()
    for i, word in enumerate(words, 1):
        unique_words.add(word)
        vocab_growth.append((i, len(unique_words)))
    return np.array(vocab_growth)

def plot_heaps_law(df, title):
    """Plots Heaps' Law based on vocabulary growth for a single comedy set."""
    transcript = df[df['title'] == title]['transcript'].values[0]
    words = tokenize_words(transcript, remove_stop_words=False)
    vocab_growth = calculate_vocabulary_growth(words)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(vocab_growth[:, 0], vocab_growth[:, 1], color='grey')
    plt.xlabel("Total Words (N)")
    plt.ylabel("Unique Words")
    plt.title(f"Heaps' Law - '{title}'")
    plt.legend()
    plt.show()


# Generate the updated DataFrame and plot


def plot_zipf_and_heaps_all_comedians(df):
    """
    Plots Zipf's Law and Heaps' Law side by side for each comedian in the dataframe.
    
    Parameters:
    - df (DataFrame): DataFrame containing comedy data with titles, lengths, and transcripts.
    """
    num_comedians = len(df)
    fig, axes = plt.subplots(num_comedians, 2, figsize=(12, 6 * num_comedians))
    
    for idx, row in df.iterrows():
        title = row['title']
        transcript = row['transcript']
        
        # Tokenize words
        words = tokenize_words(transcript, remove_stop_words=False)
        
        # Prepare Zipf's Law data
        word_data = get_word_data(words)
        word_data['rank'] = np.arange(1, len(word_data) + 1)
        log_rank = np.log10(word_data['rank'])
        log_freq = np.log10(word_data['counts'])
        
        # Perform linear regression for Zipf's Law slope
        slope, intercept, r_value, p_value, std_err = linregress(log_rank, log_freq)
        
        # Plot Zipf's Law (log-log scale)
        axes[idx, 0].loglog(word_data['rank'], word_data['counts'], marker=".", linestyle='none', color="grey")
        axes[idx, 0].set_title(title)
        axes[idx, 0].set_xlabel('Rank (log scale)')
        axes[idx, 0].set_ylabel('Frequency (log scale)')
        
        # Prepare Heaps' Law data
        vocab_growth = calculate_vocabulary_growth(words)
        
        # Plot Heaps' Law
        axes[idx, 1].plot(vocab_growth[:, 0], vocab_growth[:, 1], color='grey')
        axes[idx, 1].set_title(title)
        axes[idx, 1].set_xlabel("Total Words (N)")
        axes[idx, 1].set_ylabel("Unique Words (V)")
    
    # Adjust layout for readability
    plt.tight_layout()
    plt.show()

# innovation_rate_df = get_innovation_rate_data(df)
# plot_innovation_rate(innovation_rate_df)

# innovation_rate_df = get_innovation_rate_data_with_word_count(df)
# plot_innovation_rate_by_word_count(innovation_rate_df)

# plot_zipf_and_heaps_all_comedians(df)
# print(df['title'].unique())

# compare with The black Cat by Edgar Allan Poe
mitch_words = tokenize_words(df[df['title'] == 'MITCH HEDBERG: COMEDY CENTRAL SPECIAL (1999)']['transcript'].values[0], remove_stop_words=False)
mitch_word_data = get_word_data(mitch_words)
print(mitch_word_data.head(100))
print(calculate_innovation_rate(mitch_words))

jesus_is_magic = tokenize_words(df[df['title'] == 'SARAH SILVERMAN: JESUS IS MAGIC (2005)']['transcript'].values[0], remove_stop_words=False)
jesus_is_magic_word_data = get_word_data(jesus_is_magic)
print(jesus_is_magic_word_data.head(30))
print(calculate_innovation_rate(jesus_is_magic))

# print(df[df['length'] > 70]['title'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/robinwoodfamily/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/robinwoodfamily/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


      word  counts     probs  total_unique
0        I     241  0.055237          1117
1        a     171  0.039193          1117
2      the     123  0.028192          1117
3       to     122  0.027962          1117
4      you      79  0.018107          1117
..     ...     ...       ...           ...
95   never       8  0.001834          1117
96     now       7  0.001604          1117
97  comedy       7  0.001604          1117
98  around       7  0.001604          1117
99    need       7  0.001604          1117

[100 rows x 4 columns]
Unique words: 1117
Total words: 4363
0.25601650240660095
    word  counts     probs  total_unique
0      I     443  0.061019          1485
1    you     208  0.028650          1485
2      a     205  0.028237          1485
3     it     187  0.025758          1485
4    the     164  0.022590          1485
5   know     131  0.018044          1485
6     to     121  0.016667          1485
7   that     117  0.016116          1485
8    And      99  0.013636        